# Compute focal mechanism distribution

In [ ]:
%matplotlib inline
import os
import re
import sys
import h5py
import numpy
import pickle
import matplotlib.pyplot as plt
from openquake.mbt.oqt_project import OQtProject
from openquake.mbt.tools.geo import get_idx_points_inside_polygon
from openquake.hmtk.parsers.catalogue.gcmt_ndk_parser import ParseNDKtoGCMT

In [ ]:
def mecclass(plungt, plungb, plungp):
    """
    This is taken from the FMC package.
    See https://josealvarezgomez.wordpress.com/

    It provides a classification of the rupture mechanism based on the
    Kaverina et al. (1996) methodology.

    :parameter plungt:
    :parameter plungb:
    :parameter plungp:

    """
    plunges = numpy.asarray((plungp, plungb, plungt))
    P = plunges[0]
    B = plunges[1]
    T = plunges[2]
    maxplung, axis = plunges.max(0), plunges.argmax(0)
    if maxplung >= 67.5:
        if axis == 0:  # P max
            clase = 'N'  # normal faulting
        elif axis == 1:  # B max
            clase = 'SS'  # strike-slip faulting
        elif axis == 2:  # T max
            clase = 'R'  # reverse faulting
    else:
        if axis == 0:  # P max
            if B > T:
                clase = 'N-SS'  # normal - strike-slip faulting
            else:
                clase = 'N'  # normal faulting
        if axis == 1:  # B max
            if P > T:
                clase = 'SS-N'  # strike-slip - normal faulting
            else:
                clase = 'SS-R'  # strike-slip - reverse faulting
        if axis == 2:  # T max
            if B > P:
                clase = 'R-SS'  # reverse - strike-slip faulting
            else:
                clase = 'R'  # reverse faulting
    return clase

In [ ]:
def get_simpler(dct):
    ndct = {'N': [], 'SS': [], 'R': []}
    for key in dct.keys():
        if key == 'SS-N' or key == 'SS-R':
            ndct['SS'] += dct[key]
        elif key == 'N-SS':
            ndct['N'] += dct[key]
        elif key == 'R-SS':
            ndct['R'] += dct[key]
        else:
            ndct[key] += dct[key] 
    return ndct
    
def get_simplified_classification(histo, keys):
    simpl_class = {'N': 0, 'SS': 0, 'R': 0}
    for num, key in zip(histo, keys):
        if key == 'SS-N' or key == 'SS-R':
            simpl_class['SS'] += num
        elif key == 'N-SS':
            simpl_class['N'] += num
        elif key == 'R-SS':
            simpl_class['R'] += num
        else:
            simpl_class[key] += num 
    return simpl_class

In [ ]:
plot = False
#
# read project
project_pickle_filename = os.environ.get('OQMBT_PROJECT')
oqtkp = OQtProject.load_from_file(project_pickle_filename)
model_id = oqtkp.active_model_id
model = oqtkp.models[model_id]
prj_dir = os.path.dirname(project_pickle_filename)

#
# set hdf5 file names
gcmt_filename = os.path.join(prj_dir, model.focal_mechanisms_filename)
focal_mech_hdf5_filename = os.path.join(prj_dir, oqtkp.focal_mech_hdf5_filename)
#
# set source ID
try:
    area_source_ids_list = getattr(oqtkp,'active_source_id')
except:
    print('Active source ID not defined in the OQMBT project')
    area_source_ids_list = ['10']
#
# info
print('Active model is:', model_id)
print('Processing area source with ID:', area_source_ids_list)

## Create catalogue for the analysed area source
### Read the focal mechanism catalogue

In [ ]:
# 
# parsing the GCMT catalogue
print(gcmt_filename)
parser = ParseNDKtoGCMT(gcmt_filename)
cat_gcmt = parser.read_file()
#
# area source information
src_id = area_source_ids_list[0]
src = model.sources[src_id]
#
# sheck if the area source has a geometry
if 'polygon' in src.__dict__:
    pass
elif src_id in model.nrml_sources:
    src.polygon = model.nrml_sources[src_id].polygon
    src.name = model.nrml_sources[src_id].name
    src.source_id = model.nrml_sources[src_id].source_id
else: 
    print('The source does not have a geometry assigned')
#
# selecting earthquakes within the area source
idxs = range(0, len(cat_gcmt.data['latitude']))
tmp_idxs = get_idx_points_inside_polygon(cat_gcmt.data['longitude'], 
                                                 cat_gcmt.data['latitude'],
                                                 src.polygon.lons, 
                                                 src.polygon.lats, 
                                                 idxs,
                                                 0.0)
print('The number of earthquakes within this polygon %d' % (len(tmp_idxs)))
#
# find the focal mechanisms located within the depth interval under investigation
fidxs = []
for idx in tmp_idxs:
    if ((cat_gcmt.data['depth'][idx] < float(model.catalogue_maximum_depth)) &
        (cat_gcmt.data['depth'][idx] > float(model.catalogue_minimum_depth))):
        fidxs.append(idx)
idxs_selected_fm = fidxs
#
# info
print('The number of earthquakes within this polygon %d' % (len(tmp_idxs)))
print('The number of selected earthquakes %d' % (len(idxs_selected_fm)))

### Processing the selected earthquakes

In [ ]:
fmclassification = {}
eventfm = {}
dip_1 = {}
dip_2 = {}
strike_1 = {}
strike_2 = {}
for idx in idxs_selected_fm:
    plungeb = cat_gcmt.data['plunge_b'][idx]
    plungep = cat_gcmt.data['plunge_p'][idx]
    plunget = cat_gcmt.data['plunge_t'][idx]
    mclass = mecclass(plunget, plungeb, plungep)
    eventfm[idx] = mclass
    if mclass in fmclassification:
        fmclassification[mclass] += 1
        dip_1[mclass].append(cat_gcmt.data['dip1'][idx])
        dip_2[mclass].append(cat_gcmt.data['dip2'][idx])
        strike_1[mclass].append(cat_gcmt.data['strike1'][idx])
        strike_2[mclass].append(cat_gcmt.data['strike2'][idx])
    else:
        fmclassification[mclass] = 1
        dip_1[mclass] = [cat_gcmt.data['dip1'][idx]]
        dip_2[mclass] = [cat_gcmt.data['dip2'][idx]]
        strike_1[mclass] = [cat_gcmt.data['strike1'][idx]]
        strike_2[mclass] = [cat_gcmt.data['strike2'][idx]]

### Focal mechanisms histogram

In [ ]:
classes = ['N', 'R', 'SS', 'N-SS', 'SS-N', 'SS-R', 'R-SS']

bin_edges = numpy.array([0, 1, 2, 3, 4, 5, 6, 7])
histo = []

for key in classes:
    if key in fmclassification:
        histo.append(fmclassification[key])
    else:
        histo.append(0)

simplified = get_simplified_classification(histo, classes)
print(fmclassification)
print(simplified)

histosimple = []
for key in classes:
    if key in simplified:
        histosimple.append(simplified[key])
    else:
        histosimple.append(0)
    
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(1, 1, 1)
    
plt.bar(bin_edges[0:-1], histo, 
        width=numpy.diff(bin_edges),
        edgecolor='red', 
        facecolor='orange', 
        linewidth=3, 
        alpha=1.0,
        label='Kaverina')

plt.bar(bin_edges[0:-1], histosimple, 
        width=numpy.diff(bin_edges),
        edgecolor='blue', 
        facecolor='None', 
        linewidth=3, 
        alpha=1.0,
        label='Simplified')

plt.ylabel(r'Earthquake count', fontsize=14)
plt.grid(which='major', axis='y', linestyle='--')
plt.title('Area source %s' % area_source_ids_list[0])

be = numpy.array(bin_edges)
plt.xticks(be[0:-1]+(be[1]-be[0])/2., classes)

xlimits = plt.xlim([0, max(bin_edges)])
leg = plt.legend()

### Strike Vs. Dip distribution for each rupture mechanism class

In [ ]:
KAVERINA = {'N'   : 'blue',
            'SS'  : 'green',
            'R'   : 'red',
            'N-SS': 'turquoise',
            'SS-N': 'palegreen',
            'R-SS': 'goldenrod',
            'SS-R': 'yellow' }

In [ ]:
import matplotlib.gridspec as gridspec

if len(fmclassification):
    fs = 14
    fig = plt.figure(figsize=(12, 10))
    gs = gridspec.GridSpec(4, 2, wspace=0.0, hspace=0.0)
    for key, igs in zip(classes, range(0, len(classes))):
        ax = plt.subplot(gs[igs])
        if key in strike_1:
            plt.plot(strike_1[key], dip_1[key], 'o', markersize=8, color=KAVERINA[key])
            plt.plot(strike_2[key], dip_2[key], 'o', markersize=6, color=KAVERINA[key], alpha=0.5, markeredgecolor='blue')
        plt.xlim([0, 360])
        plt.ylim([0, 90])
        plt.grid(which='major', )

        x = numpy.arange(30, 90, 30)
        ax.set_yticks(x)
        x = numpy.arange(30, 360, 30)
        ax.set_xticks(x)
        if igs in [0, 1, 2, 3, 4]:
            ax.set_xticklabels([])
        else:
            ax.set_xlabel('strike', fontsize=fs)
        if igs in [1, 3, 5]:
            ax.set_yticklabels([])
        else:
            ax.set_ylabel('dip', fontsize=fs)
        plt.text(.05,.90,key,
            horizontalalignment='left',
            transform=ax.transAxes)

In [ ]:
if len(fmclassification):
    stk1 = get_simpler(strike_1)
    stk2 = get_simpler(strike_2)
    dip1 = get_simpler(dip_1)
    dip2 = get_simpler(dip_2)

In [ ]:
if len(fmclassification):
    fs = 14
    fig = plt.figure(figsize=(12, 8))
    gs = gridspec.GridSpec(3, 1, wspace=0.0, hspace=0.0)
    for key, igs in zip(dip1.keys(), range(0, len(dip1.keys()))):
        ax = plt.subplot(gs[igs])
        if key in dip1.keys():
            plt.plot(stk1[key], dip1[key], 'o', markersize=8, color=KAVERINA[key])
            plt.plot(stk2[key], dip2[key], 'o', markersize=6, color=KAVERINA[key], alpha=0.5, markeredgecolor='blue')
        plt.xlim([0, 360])
        plt.ylim([0, 90])
        plt.grid(which='major', )

        x = numpy.arange(30, 90, 30)
        ax.set_yticks(x)
        x = numpy.arange(30, 360, 30)
        ax.set_xticks(x)
        if igs in [0, 1]:
            ax.set_xticklabels([])
        else:
            ax.set_xlabel('strike', fontsize=fs)
        ax.set_ylabel('dip', fontsize=fs)
        plt.text(.05,.90,key,
            horizontalalignment='left',
            transform=ax.transAxes)

### Plot focal mechanisms map

In [ ]:
if len(fmclassification) and plot:
    import cartopy
    import cartopy.crs as ccrs
    import matplotlib.patheffects as PathEffects
    from obspy.imaging.beachball import beach

    fig = plt.figure(figsize=(16, 10), dpi=300)
    ax = plt.axes(projection=ccrs.PlateCarree())
    gl = ax.gridlines(crs=ccrs.PlateCarree(), 
                      draw_labels=True,
                      linewidth=2, color='gray', alpha=0.5, linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_left = False

    dlt = 2
    limits = [min(src.polygon.lons)-dlt, min(src.polygon.lats)-dlt, 
              max(src.polygon.lons)+dlt, max(src.polygon.lats)+dlt]
    tmp = ax.coastlines()
    #ax.add_feature(cartopy.feature.OCEAN, zorder=0)
    # set the area for the plot
    tmp = ax.set_extent([limits[0], limits[2], limits[1], limits[3]], ccrs.Geodetic())

    # catalogue
    ax.plot(cat_gcmt.data['longitude'], cat_gcmt.data['latitude'], 's',
            linewidth=1, alpha=0.4, transform=ccrs.Geodetic(), color='green',
            path_effects=[PathEffects.withStroke(linewidth=3, foreground="g")]) 
    ax.plot(src.polygon.lons, src.polygon.lats, '-', linewidth=3, color='blue', transform=ccrs.Geodetic())
    ax.plot(cat_gcmt.data['longitude'][idxs_selected_fm], 
            cat_gcmt.data['latitude'][idxs_selected_fm], 'o', linewidth=3, color='blue', transform=ccrs.Geodetic())

    # 
    for idx, lon, lat, mag in zip(idxs_selected_fm,
                                    cat_gcmt.data['longitude'][idxs_selected_fm],
                                    cat_gcmt.data['latitude'][idxs_selected_fm],
                                    cat_gcmt.data['magnitude'][idxs_selected_fm]):

        eve = cat_gcmt.gcmts[idx]
        com = eve.moment_tensor._to_6component()
        try: 
            bcc = beach(com, xy=(lon, lat), 
                        width=eve.magnitude*0.1,
                        linewidth=1, 
                        zorder=20, 
                        size=mag,
                        facecolor=KAVERINA[eventfm[idx]])
            bcc.set_alpha(0.8)
            bcc.set_label(eventfm[idx])
            ax.add_collection(bcc)
        except:
            print('Error in the moment tensor?', com)
            continue
    #
    for lab in classes:
        circ1 = plt.plot(limits[0]-0.1, 
                         limits[1]-0.1, 
                         linestyle="none", 
                         marker="o", 
                         alpha=1.0, 
                         markersize=10.0, 
                         markerfacecolor=KAVERINA[lab], 
                         label=lab)
    # Legend
    leg = plt.legend()
    # Grid
    grd = ax.gridlines()

## CSV files

In [ ]:
if len(fmclassification):
    #
    # set the csv file name
    aa = re.sub('\.hdf5', '', os.path.basename(focal_mech_hdf5_filename))
    csv_filename = '{:s}-{:s}-{:s}.csv'.format(aa, model_id, src_id)
    print(csv_filename)
    #
    #
    path = os.path.join(oqtkp.directory, 'focal_mechs')
    if not os.path.exists(path):
        os.makedirs(path)
        print ('Creating folder: {:s}'.format(path))
    else:
        print ('Folder {:s} exists'.format(path))
    #
    # Writing csv file
    outfile = os.path.join(path, csv_filename)
    print ('Writing {:s}'.format(outfile))
    fou = open(outfile, 'w')
    fou.write('strike,dip,rake,weight\n')
    fou.write('{:.2f},{:.2f},{:.2f},{:.2f}\n'.format(0, 90, 0, 1.0))
    fou.close()

## HDF5 files
### Updating the hdf5 file with focal mechanisms data

In [ ]:
if len(fmclassification):
    print('Writing', focal_mech_hdf5_filename)
    fhdf5 = h5py.File(focal_mech_hdf5_filename,'a')
    src_id = area_source_ids_list[0]

    # Update/create model group
    if model_id in fhdf5.keys():
        print('Group exists. Set group %s' % (model_id))
        grp = fhdf5[model_id]
    else:
        print('Create group: %s' % (model_id))
        grp = fhdf5.create_group(model_id)

    # Update/create source group
    if src_id in fhdf5[model_id].keys():
        print('Group exists. Set group %s' % (src_id))
        grpsrc = fhdf5[model_id][src_id]
    else:
        print('Create group: %s' % (src_id))
        grpsrc = fhdf5[model_id].create_group(src_id)

    # Update/create datasets
    dset_ids = ['kaverina_histogram', 'simplified_histogram']
    for dset_id in dset_ids:
        if dset_id in grpsrc:
            del grpsrc[dset_id]

    dataset = grpsrc.create_dataset('kaverina_histogram', data=histo)
    dataset.attrs['labels'] = '-'.join(['%s' % lab for lab in classes])
    dataset = grpsrc.create_dataset('simplified_histogram', data=histosimple)
    dataset.attrs['labels'] = '-'.join(['%s' % lab for lab in classes])

    strike_simpl_1 = stk1
    strike_simpl_2 = stk1
    dip_simpl_1 = dip1
    dip_simpl_2 = dip2

    for tpe in ['strike_1', 'strike_2', 'dip_1', 'dip_2',
               'strike_simpl_1', 'strike_simpl_2', 'dip_simpl_1', 'dip_simpl_2']:

        # Update/create source group
        if tpe in fhdf5[model_id][src_id].keys():
            print('Group exists. Set group %s' % (tpe))
            grpsrc = fhdf5[model_id][src_id][tpe]
            # cleaning
            for dset_id in grpsrc:
                del grpsrc[dset_id]   
        else:
            print('Create group: %s' % (tpe))
            grpsrc = fhdf5[model_id][src_id].create_group(tpe)

        tdct = eval(tpe)
        for key in tdct.keys():
            dataset = grpsrc.create_dataset(key, data=tdct[key])

    fhdf5.close()

### Initialising the hdf5 containing the nodal plane distribution

In [ ]:
if len(fmclassification):
    #
    # open the hdf5 containing the nodal plane information
    nodal_plane_dist_filename = os.path.join(oqtkp.directory, model.nodal_plane_dist_filename)
    fhdf5 = h5py.File(nodal_plane_dist_filename,'a')
    #
    # add the dataset for the current area source, if missing
    if src_id in fhdf5.keys():
        print ('Datasets already available')
    else:
        print ('Adding dummy dataset')
        x = numpy.zeros(1, dtype=[('strike','f4'),('dip', 'f4'), ('rake', 'f4'), ('wei', 'f4')])
        dset = fhdf5.create_dataset(src_id, data=x)
    fhdf5.close()